In [16]:
from collections import defaultdict
import pandas as pd
from urllib.parse import unquote
import tqdm
import os
from bs4 import BeautifulSoup  
import numpy as np
import urllib
import re
import urllib.parse
import time
import scipy
import scipy.linalg

In [6]:
path = 'raw/'

In [7]:
re.findall(r'/wiki/+\w*,?\b','/wiki/fsd_de_ded ddfdfsdsd sds ds sdsd /wiki/dfdf')

['/wiki/fsd_de_ded', '/wiki/dfdf']

In [8]:
topics = {}
for i, file in enumerate(os.listdir(path)):
    topics[file[:-5]] = i

In [9]:
len(topics)

2379

In [10]:
def get_matrix_re():
    matrix = np.zeros((len(os.listdir(path)), len(os.listdir(path))))
    df = defaultdict(lambda: [])
    for i in tqdm.tqdm_notebook(os.listdir(path)):
        with open (path+i, encoding="UTF-8") as file:
            read = unquote(file.read())
            cur_topic = i[:-5]
            link_site = re.findall(r'/wiki/.+?\b',read)
            for link in link_site:
                if link in topics.keys():
                    df[i.split('.')[0]].append(link.split('/')[-1])
                    matrix[topics[cur_topic]][topics[link[6:]]] = 1         
    return matrix

In [11]:
def get_matrix():
    matrix = np.zeros((len(os.listdir(path)), len(os.listdir(path))))
    for file in tqdm.tqdm_notebook(os.listdir(path)):
        with open(f"raw/{file}", encoding='utf-8') as f:
            cur_topic = file[:-5]
            bs = BeautifulSoup(f.read(), "html.parser")
            all_a = bs.find_all("a")
            for link in all_a:
                if 'href' in link.attrs:
                    href = urllib.parse.unquote(link['href'])
                    if href.startswith("/wiki/") and href[6:] in topics:
                        matrix[topics[cur_topic]][topics[href[6:]]] = 1
    return matrix

In [12]:
def process(matrix):
    for i in matrix:
        for j in range(len(i)):
            if i[j] == 0:
                i[j] = 0.001
    row_sums = matrix.sum(axis=1)
    norm_matrix = matrix / row_sums[:, np.newaxis]
    vector = np.array([0.2]*len(os.listdir(path)))
    for _ in range(100):
        vector = norm_matrix.transpose().dot(vector)
    # np.take(vector, [1, 2])
    rank = {}
    for i, value in enumerate(vector):
        rank[i] = value
    return rank

In [13]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{os.listdir(path)[index][:-5]}: {value}")

In [24]:
def HITS(matrix, word):
    page_with_word = []

    for i, name in enumerate(files_name):
        if word.lower() in name.lower():
            page_with_word.append(i)

    base_set = page_with_word.copy()
    for i in page_with_word:
        for j in range(0, len(files_name)):
            if matrix[i][j] > 0:
                base_set.append(j)

        for j in range(0, len(files_name)):
            if matrix[j][i] > 0:
                base_set.append(j)

    base_set = list(set(base_set))
    a = np.zeros((len(base_set), len(base_set)))
    for i in range(0, len(base_set)):
        for j in range(0, len(base_set)):
            a[i, j] = matrix[base_set[i]][base_set[j]]

    at = a.transpose()

    aat = a.dot(at)
    ata = at.dot(a)

    hub = np.abs(np.linalg.eigh(aat)[1]).sum(axis=1)
    auth = np.abs(np.linalg.eigh(ata)[1]).sum(axis=1)

    print("\n hub \n")

    dhub = {}

    for i, value in enumerate(hub):
        dhub[i] = value
    n = 10
    dbase = {}
    for i, value in enumerate(base_set):
        dbase[value] = i

    new_dhub = {}
    for i in page_with_word:
        new_dhub[dbase[i]] = dhub[dbase[i]]

    items = sorted(new_dhub.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(files_name[base_set[index]] + ' ' + str(value))

    print("\n auth \n")


    dauth = {}

    for i, value in enumerate(auth):
        dauth[i] = value

    new_dauth = {}
    for i in page_with_word:
        new_dauth[dbase[i]] = dauth[dbase[i]]

    items = sorted(new_dauth.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(files_name[base_set[index]] + ' ' + str(value))

In [37]:
matrix = get_matrix()

In [38]:
rank = process(matrix)

In [39]:
page_rank(rank=rank)

Английский_язык: 6.710244106349573
США: 4.66682684199059
Франция: 3.891332065960388
Великобритания: 3.755379111713333
Германия: 3.0454025051563165
Соединённые_Штаты_Америки: 2.982485368742181
Италия: 2.41149748041042
СССР: 2.385908911034225
Викисклад: 2.3083234328050564
Латинский_язык: 2.3054123310637404


In [40]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.21913073924573098
Армстронг,_Нил_Олден: 0.0820416765113164
Армстронг,_Нил: 0.07772736651558804
Армстронг,_Билли_Джо: 0.06033081219855844
Армстронг,_Алан: 0.05467384689837618
Армстронг_(фамилия): 0.054339929130478845
Армстронг,_Эдвин: 0.04872429424522622
Армстронг,_Лэнс: 0.04765582655907486
Билли_Джо_Армстронг: 0.04497463456555457
Армстронг,_Крэйг: 0.03901933675387288


In [25]:
HITS(matrix, 'армстронг')


hub

Армстронг,_Генри_(боксёр) 29.42320429693974
Армстронг,_Джо 28.773862717012392
Армстронг,_Чарльз 27.98399650256239
Армстронг,_Джеймс_(борец) 27.30465434018304
Армстронг,_Джеймс 27.10400738542007
Армстронг,_Уильям_Лестер 27.080429851326862
Армстронг_(лунный_кратер) 26.891574277992305
Армстронг,_Дженни 26.859611655210056
Армстронг,_Уильям_Джордж 26.48752477383553
Армстронг,_Джонас 26.30655190752678

 auth 

Армстронг_(лунный_кратер) 30.212507467207907
Армстронг,_Кёрли 29.945353033123723
Армстронг,_Скотт_(баскетболист) 29.400616317359717
Армстронг 28.71749898795197
Армстронг,_Нил_Олден 28.70426439274828
Клан_Армстронг 28.197415045072596
Армстронг,_Даррелл 27.882247564968267
Армстронг,_Джерри 27.84761242731537
Билли_Джо_Армстронг 27.45050772426282
Армстронг,_Билли_Джо 27.27720388130315
